# Quarter-Mile Race from Standing Start: RWD vs AWD
This notebook simulates a standing-start quarter-mile race (402.336 m) between two cars with configurable advanced powertrains.
## What you can customize per car
- Powertrain type (`ICE`, `BEV`)
- Weight (`mass`), drivetrain (`RWD` or `AWD`), tires, and aero
- ICE torque curve, gear ratios, final drive, and shift behavior
- Manual shift time with zero wheel power during shifts
- Powertrain and driveline efficiencies so wheel power is always below source power
The notebook includes:
- Acceleration behavior comparison
- Speed vs Time plot
- Distance vs Time plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import all simulation logic from the reusable module.
# quarter_mile_sim.py lives alongside this notebook, which makes it
# importable in CPython, JupyterLab, and JupyterLite (Pyodide / WASM).
from quarter_mile_sim import (
    make_car,
    simulate_quarter_mile,
    interp_curve,
    QUARTER_MILE_M,
    DEFAULT_DT,
)

# Notebook-level aliases kept for readability in the cells below
quarter_mile_m = QUARTER_MILE_M
dt = DEFAULT_DT


In [ ]:
# Single place to customize both cars using a consistent typed schema
# Car: name + vehicle + powertrain
# Powertrain: type + driving_axles + motors + gearbox + efficiency
# Motor: rpm bounds + torque curve
# Gearbox: type + ratios/shift rules

car_specs = {
    "RWD Car": {
        "name": "RWD Car",
        "vehicle": {
            "mass": 1500,
            "CdA": 0.66,
            "wheel_radius_m": 0.335,
            "rolling_resistance": 0.015,
            "tire": {
                "width_mm": 300,
                "compound": "summer",
                "base_mu": 1.10
            }
        },
        "powertrain": {
            "type": "ICE",
            "driving_axles": "RWD",
            "efficiency": {
                "engine": 1.0,
                "driveline": 0.90
            },
            "motors": [
                {
                    "name": "ICE Engine",
                    "min_rpm": 900,
                    "max_rpm": 7200,
                    "torque_curve_rpm_nm": [[1000, 380], [2000, 570], [3000, 860], [4000, 850], [5000, 850], [6000, 800], [7000, 690], [7300, 650], [8000,100]]
                }
            ],
            "gearbox": {
                "type": "manual",
                "gear_ratios": [3.10, 2.10, 1.55, 1.22, 1.00, 0.82],
                "final_drive": 3.73,
                "launch_rpm": 2800,
                "shift_rpm": 6900,
                "shift_time_s": 0.30
            }
        }
    },
    "AWD Car": {
        "name": "AWD Car",
        "vehicle": {
            "mass": 2400,
            "CdA": 0.68,
            "wheel_radius_m": 0.350,
            "rolling_resistance": 0.016,
            "tire": {
                "width_mm": 300,
                "compound": "summer",
                "base_mu": 1.05
            }
        },
        "powertrain": {
            "type": "BEV",
            "driving_axles": "AWD",
            "efficiency": {
                "motor": 1.0,
                "inverter": 0.96
            },
            "motors": [
                {
                    "name": "Combined eMotors",
                    "min_rpm": 0,
                    "max_rpm": 16000,
                    "torque_curve_rpm_nm": [[0, 850], [6000, 850], [7000, 810], [8000, 660], [10000, 520], [12000, 420], [14000, 340], [16000, 290]]
                }
            ],
            "gearbox": {
                "type": "single_speed",
                "ratio": 9.0
            }
        }
    }
}

In [ ]:
# Build runtime car objects from the typed spec schema defined above.
# Physics, drivetrain, and integration logic lives in quarter_mile_sim.py.
cars = {name: make_car(name, spec) for name, spec in car_specs.items()}


In [ ]:
results = {name: simulate_quarter_mile(spec, dt=dt) for name, spec in cars.items()}
for name, r in results.items():
    car = cars[name]
    summary = f"{name} | {car['powertrain_type']} | {car['drivetrain']} | {car['tire_width_mm']:.0f}mm {car['tire_compound']}"
    if car['powertrain_type'] == 'ICE':
        summary += f" | {car['ice']['gearbox_type']} {len(car['ice']['gear_ratios'])}spd | shifts={r['shift_count']}"
    print(summary)
    print(f"  mass={car['mass']:.0f}kg -> ET={r['elapsed_time']:.2f}s, trap={r['trap_speed']*3.6:.1f} km/h")
winner = min(results.items(), key=lambda kv: kv[1]['elapsed_time'])[0]
print(f"\nWinner over quarter mile: {winner}")

In [ ]:
# 1) Acceleration behavior and wheel torque from time-simulated samples
results = {name: simulate_quarter_mile(spec, dt=dt) for name, spec in cars.items()}
fig, ax1 = plt.subplots(figsize=(10, 5))
color1 = 'tab:blue'
ax1.set_xlabel('Speed (km/h)')
ax1.set_ylabel('Acceleration (m/s²)', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
for name, r in results.items():
    speed_kph = r['speed'] * 3.6
    accel = r['accel']
    order = np.argsort(speed_kph)
    ax1.plot(speed_kph[order], accel[order], label=name, linewidth=2)
ax2 = ax1.twinx()
color2 = 'tab:red'
ax2.set_ylabel('Wheel Torque (Nm)', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
for name, r in results.items():
    speed_kph = r['speed'] * 3.6
    wheel_torque = r['wheel_torque']
    order = np.argsort(speed_kph)
    ax2.plot(speed_kph[order], wheel_torque[order], label=f"{name} (torque)", linestyle='--', linewidth=2)
ax1.set_title("Acceleration and Wheel Torque vs Speed (simulated)")
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# 2) Power curves (HP vs RPM) for both cars
fig, ax = plt.subplots(figsize=(10, 5))
for name, car in cars.items():
    if car['powertrain_type'] == 'ICE':
        rpm_range = np.linspace(0, car['ice'].get('redline_rpm', 7000), 150)
        torques = np.array([interp_curve(car['ice']['torque_curve_rpm_nm'], rpm) for rpm in rpm_range])
        hp = (torques * rpm_range) / 7745
        ax.plot(rpm_range, hp, label=name, linewidth=2)
    elif car['powertrain_type'] == 'BEV':
        rpm_range = np.linspace(0, car['motor'].get('max_rpm', 16000), 150)
        torques = np.array([interp_curve(car['motor']['torque_curve_rpm_nm'], rpm) for rpm in rpm_range])
        hp = (torques * rpm_range) / 7745
        ax.plot(rpm_range, hp, label=name, linewidth=2)
ax.set_xlabel('RPM')
ax.set_ylabel('Power (HP)')
ax.set_title('Power Curve vs RPM')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 3) Race plots together: distance vs time and speed vs time
fig, axes = plt.subplots(1, 2, figsize=(12, 4.5))
for name, r in results.items():
    axes[0].plot(r['time'], r['distance'], label=name)
    axes[1].plot(r['time'], r['speed'] * 3.6, label=name)
axes[0].axhline(quarter_mile_m, linestyle='--', linewidth=1.0, color='k', alpha=0.7, label='Quarter mile')
axes[0].set_title("Distance vs Time")
axes[0].set_xlabel("Time (s)")
axes[0].set_ylabel("Distance (m)")
axes[0].grid(True, alpha=0.3)
axes[0].legend()
axes[1].set_title("Speed vs Time")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Speed (km/h)")
axes[1].grid(True, alpha=0.3)
axes[1].legend()
plt.tight_layout()
plt.show()

## Notes
- Edit `car_specs` in Cell 3 using the typed schema: `Car -> Powertrain -> Motor/Gearbox`.
- `powertrain.driving_axles` is where `RWD` / `AWD` is defined.
- `powertrain.type` is where `ICE` / `BEV` is defined.
- Manual shifts use a zero-wheel-power shift window (`gearbox.shift_time_s`).
- Wheel power includes efficiency losses and is lower than source power.